In [1]:
import keras
import efficientnet.keras as efn 
from keras.models import load_model
from keras.preprocessing import image
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Using TensorFlow backend.
W0809 17:20:25.157788 140735510987648 deprecation.py:323] From /Users/mananmehta/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Object Detection to detect T-shirts and crop them from images
saved_model_dir = './fashionpedia-api/model'  # specify the model dir here


session = tf.Session(graph=tf.Graph())
_ = tf.saved_model.loader.load(session, ['serve'], saved_model_dir)

W0809 17:20:27.352929 140735510987648 deprecation.py:323] From <ipython-input-2-61b31bc823f7>:6: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


In [3]:
# Loading trained siamese triplet Model
model = load_model('/Users/mananmehta/Downloads/model2_epoch_2_max_pooling_with_shallow_layers_lr_0.3.h5')
model._make_predict_function()

W0809 17:20:42.774966 140735510987648 deprecation.py:506] From /Users/mananmehta/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
/Users/mananmehta/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def check_t_shirt_site(path):
    '''
    Checks whether an image contains T-Shirt or not
    '''
    with open(path,'rb') as f:
        np_image_string = np.array([f.read()])
    
    num_detections, detection_classes, detection_scores, image_info = session.run(['NumDetections:0','DetectionClasses:0', 'DetectionScores:0','ImageInfo:0']
                                                                                      ,feed_dict = {'Placeholder:0':np_image_string})
    num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
    detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
    detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections] 
    
    for i in range(num_detections):
        if detection_classes[i] == 2 and detection_scores[i] > 0.90 : 
            return True
    return False

In [10]:
# returns embedding matrix for a folder of images
def generate_embedding_matrix_sites(path):
    '''
    Generates embedding matrix for site images, of type (img_path,embedding)
    '''
    embeddings = []
    for file in os.listdir(path):
        if check_t_shirt_site(path + '/' + file):
            
            img = image.load_img(path + '/' + file,target_size = (300,225))
            img = image.img_to_array(img)
            img = img/255
            img = np.expand_dims(img,axis = 0)
            embedding = model.predict(img)
            embedding = np.squeeze(embedding)
            embeddings.append([path + '/' + file,embedding])
    return np.array(embeddings)   
        

In [6]:
ontology = json.load(open('./fashionpedia-api/data/demo/category_attributes_descriptions.json'))
def check_t_shirt(path):
        '''
        Checks whether a Trendy image contains T-Shirt or not.
        If yes it returns cropped portion of image, containing T-shirt
        '''
    with open(path,'rb') as f:
        np_image_string = np.array([f.read()])
    
    num_detections, detection_boxes, detection_classes, detection_scores, image_info = session.run(['NumDetections:0','DetectionBoxes:0','DetectionClasses:0', 'DetectionScores:0','ImageInfo:0']
                                                                                      ,feed_dict = {'Placeholder:0':np_image_string})
    num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
    detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
    detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
    detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
    
    for i in range(num_detections):
        if detection_classes[i] == 2 and detection_scores[i] > 0.90 : 
            top,left,bottom,right = detection_boxes[i]
            bbox = (left,top,right,bottom)
            return (True,bbox)
    return (False,(0,0,0,0))

In [7]:
def generate_embedding_matrix_trendy(path):
    '''
    Generates Embedding matrix of trendy images
    '''
    embeddings = []
    for file in os.listdir(path):
        is_t_shirt, bbox = check_t_shirt(path + '/' + file)
        
        if is_t_shirt == False:
            continue

        img = image.load_img(path + '/' + file)
        img = img.crop(bbox)
        img = img.resize((225,300))
        img = image.img_to_array(img)
        img = img/255
        img = np.expand_dims(img,axis = 0)
        embedding = model.predict(img)
        embedding = np.squeeze(embedding)
        embeddings.append([path + '/' + file,embedding])
        
    return np.array(embeddings)       
            

In [8]:
def distance(a,b):
    # Calculates distance between two embeddings
        return np.sqrt(np.sum((a-b)**2))
    
def find_top_neighbours(trendy_embedding_matrix,site_embedding_matrix):
    '''
    Finds top 5 nearest neighbours for each trendy image
    '''
    top_hits = []
    for trendy_emb in trendy_embedding_matrix:
        distances = []
        for site_emb in site_embedding_matrix:
            distances.append((site_emb[0],distance(trendy_emb[1],site_emb[1])))
            
        distances = sorted(distances,key = lambda x : x[1])[:5]
        top_hits.append([trendy_emb[0],distances])

    return top_hits
        

In [12]:
#trend = generate_embedding_matrix_trendy('/Users/mananmehta/Desktop/dataset')
#sites = generate_embedding_matrix_sites('/Users/mananmehta/Desktop/dataset 2')

/Users/mananmehta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [13]:
#hello = find_top_neighbours(trend,sites)

In [15]:
#axes = []
#fig = plt.figure(figsize = (20,20))
#c = 0
#for i in range(len(hello)):
#    img = image.load_img(hello[i][0],target_size = (300,225))
#    axes.append(fig.add_subplot(14,6,i+c+1))
#    plt.imshow(img)
#    for j in range(len(hello[i][1])):
#        axes.append(fig.add_subplot(14,6,i+j+2+c))
#        img = image.load_img(hello[i][1][j][0],target_size = (300,225))
#        plt.imshow(img)
#    c += 5
#plt.show()